# Week 1 - setup, sequencing QC, mapping, variant calling

## Setup
Due to a couple of problems along the way, the whole setup is summarized in cheatsheets. This includes Miniconda3, packages, Nextflow and Singularity installations.

## Sequencing Quality Control
* Download the fasta compressed file (MN908947.fasta.zip) containing Wuhan-1 reference sequence into working directory
* Activate MOOC environment

In [ ]:
$ conda activate MOOC

* Download the sequence data for following samples using:

In [ ]:
$ fastq-dump --split-files ERR5743893

* Create a folder to store QC reports

In [ ]:
$ mkdir -p QC_Reports

* Run fastqc on the two fastq files we just downloaded

In [ ]:
$ fastqc ERR5743893_1.fastq ERR5743893_2.fastq --outdir QC_Reports

* FastQC creates an .html report for each file. To have it summarized in one report we use a tool called multiQC.

In [ ]:
$ multiqc .

* Open the multiqc_report.html in the web browser and explore the output

## Mapping sample sequence to reference
Steps:
1. Indexing reference genome - fasta.fai
2. Map sequences - .sam
3. Compress to binary file - .bam
4. Sort the .bam for quicker downstream processing - (sorted).bam
5. Index sorted.bam file - (sorted).bam.bai
6. *Visualize in Integrative Genomics Viewer (IGV)

1. Indexing reference genome

In [ ]:
# be organized
$ mkdir Mapping

#index
$ bwa index MN908947.fasta
# or this, since this produces the fasta.fai file for IGV
$ samtools faidx MN908947.fasta

2. Map sequences

In [ ]:
$ bwa mem MN908947.fasta ERR5743893_1.fastq ERR5743893_2.fastq > Mapping/ERR5743893.sam

* Change to Mapping and check file size

In [ ]:
$ cd Mapping
$ ls -lhrt

3. Compress .sam to .bam

In [ ]:
$ cd ..
$ samtools view -@ 20 -S -b Mapping/ERR5743893.sam > Mapping/ERR5743893.bam # can change nr of threads as to specs

* Check again the file size to confirm compression

4. Sort the .bam file

In [ ]:
$ samtools sort -@ 32 -o Mapping/ERR5743893.sorted.bam Mapping/ERR5743893.bam

5. Index the (sorted).bam file

In [ ]:
$ samtools index Mapping/ERR5743893.sorted.bam

6. *Visualize in IGV<br>
  Web software: Genome > load refseq.fasta & refseq.fasta.fai<br>
  Tracks > load sampleseq.sorted.bam & sampleseq.sorted.bam.bai

## Variant calling
* Call variants from .bam file with FreeBayes. This results in .vcf file.

In [ ]:
$ freebayes -f MN908947.fasta Mapping/ERR5743893.sorted.bam  > ERR5743893.vcf

* It's good practive to compress and index the VCF file to take up less space. Most tools will accept compressed VCF files so there's no need to uncompress them later.

In [ ]:
$ bgzip ERR5743893.vcf # index
$ tabix ERR5743893.vcf.gz # compress

* examples on how to quickly retrieve variant calls:

In [ ]:
$ bcftools stats ERR5743893.vcf.gz > ERR5743893_stat.vcf.txt
# or
$ bcftools view -o output_snps.vcf.gz -O z -v snps,indels ERR5743893.vcf.gz
$ bcftools query -f '%TYPE\n' ERR5743893.vcf.gz | sort | uniq -c

## Sources
* Wellcome Connecting Science course on FutureLearn: "Bioinformatics for Biologists: Analysing and interpreting genomics datasets"
* https://github.com/CDCgov/datasets-sars-cov-2
* http://www.bioinformatics.babraham.ac.uk/projects/fastqc/
* https://multiqc.info/
* https://igv.org/app/

## More info
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5860075/
* https://dnatech.genomecenter.ucdavis.edu/faqs/when-should-i-trim-my-illumina-reads-and-how-should-i-do-it/
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3137218/